In [9]:
import rasterio
import dask.array as da
from dask_rasterio import read_raster, write_raster
import os
import fnmatch
import numpy as np
import geopandas as gpd
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy.spatial as es
import seaborn as sns
import matplotlib.pyplot as plt
import earthpy.plot as ep
import imageio
import rasterstats

In [10]:
# Directory where storms are located.
input_dir = r'Z:\LWI_StageIV\test\cropped'
output_dir= r'Z:\LWI_StageIV\test\cropped\iterate'
movie_dir = 'Z:\LWI_StageIV\!Accumulated'

# Function inputs
crop_shp = gpd.read_file(r"Z:\GIS\StageIv Boundary.shp")
la_shp = gpd.read_file("Z:\GIS\Louisiana.shp")
# dst_crs = 'EPSG:4326'
img_dir = os.path.join(output_dir, 'img')
stormName = input_dir.split('\\')[-2]
movieFilename = os.path.join(movie_dir, f'{stormName}-accum.gif')
accumRaster = os.path.join(movie_dir, f'{stormName}-cropped.tif')

In [11]:
def crop_raster(src, crop_shp):
    cropped_raster, cropped_meta = es.crop_image(src, crop_shp)
    array = cropped_raster
    array[array==0] = np.nan    
    array[array==9999] = np.nan
    src.close()
    return array

def create_image(array, title, img_dir, crop_shp, la_shp, bigMax):
    img_filename = img_dir + f'\\{title}.png'
    fig, ax = plt.subplots(figsize=(20, 15))
    
    bbox = la_shp.total_bounds
    la_extent=bbox[[0,2,1,3]]
    bbox_lwi = crop_shp.total_bounds
    lwi_extent = bbox_lwi[[0,2,1,3]]
    array[array==9999] = np.nan
    im = plt.imshow(array, extent=lwi_extent, cmap='rainbow', vmin=0, vmax=bigMax)
    cb = plt.colorbar(im, shrink=.6)
    ax.set(title=title)
    ax.set_axis_off()
    la_shp.boundary.plot(ax=plt.gca(), color='darkgrey')
    plt.savefig((img_filename))
    plt.close()


In [14]:
#Get max accumulated cropped raster value.
stats = rasterstats.zonal_stats(crop_shp, accumRaster, stats=['max'])
bigMax = stats[0]['max']
#  Get all files with extension .01h (the default pattern for uncompressed 1hr precip data from UCAR EOL).
# Script is not accounting for potential naming patterns for AK and PR.
merge_dir=[]
for filename in fnmatch.filter(os.listdir(input_dir),'*.01h'): 
    merge_dir.append((os.path.join(input_dir, filename)))

merge_dir.sort()

with rasterio.open(merge_dir[0]) as src:
        profile=src.profile
firstRasterTitle = merge_dir[0].split('\\')[-1]
hours = 0
map2array=[]
for raster in merge_dir:
    hours += 1
    outputFilename = raster.split('\\')[-1]
    title = f"{firstRasterTitle} to {outputFilename} Cumulative Precip (mm) {hours} (hr)"
    # print(f'adding {raster} to dask array list: map2array')
    src = rasterio.open(raster)
    array = src.read(1)
    # array = crop_raster(src, crop_shp)
    array[array==0] = np.nan    
    array[array==9999] = np.nan
    map2array.append(da.from_array(array, chunks=array.shape))
    if len(map2array) == 1:
        # First array element, dont accumulate.
        # Output Image
        sns.set_style("white")
        sns.set(font_scale=1.5)
        # Check whether the image and projection paths exists.
        # Create a new directory if not isExists.
        isExist_out = os.path.exists(output_dir)
        if not isExist_out:
            os.makedirs(output_dir)
        isExist_img = os.path.exists(img_dir)
        if not isExist_img:
            os.makedirs(img_dir)
        # Crop projected_raster
        # array = crop_raster(src, crop_shp)
        # Create and save Image
        create_image(array, title, img_dir, crop_shp, la_shp, bigMax)
    else: 
        # Accumulate and save Iterative Raster.
        ds_stack = da.stack(map2array)
        print (f'writing Iterative Accumulation Raster to {output_dir}\\{outputFilename}.tif')
        write_raster(f'{output_dir}\\{outputFilename}-accum.tif', da.nansum(ds_stack,0), **profile)
        # Crop Raster
        src_accum = rasterio.open(f'{output_dir}\\{outputFilename}-accum.tif')
        # array_accum = crop_raster(src_accum, crop_shp)
        array_accum = src_accum.read(1)
        array_accum[array_accum==0] = np.nan    
        array_accum[array_accum==9999] = np.nan
        # Create and Save Image
        create_image(array_accum, title, img_dir, crop_shp, la_shp, bigMax)
        # Empty Array
        map2array = []
        # Add the new iterative accumulation raster to first element of the array.
        map2array.append(da.from_array(array_accum, chunks=array_accum.shape))
        # The next iteration of the 'for raster in merge_dir:' loop will add the second element to make the next iterative accumulation.
        
        

C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:19: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.


writing Iterative Accumulation Raster to Z:\LWI_StageIV\test\cropped\iterate\ST4.2005082001.01h.tif


C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:19: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.


writing Iterative Accumulation Raster to Z:\LWI_StageIV\test\cropped\iterate\ST4.2005082002.01h.tif


C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:19: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.


...

C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:19: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.


writing Iterative Accumulation Raster to Z:\LWI_StageIV\test\cropped\iterate\ST4.2005082123.01h.tif


C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:19: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.


writing Iterative Accumulation Raster to Z:\LWI_StageIV\test\cropped\iterate\ST4.2005082200.01h.tif


C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:19: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.


In [13]:
# Create Movie
print (f'Creating gif movie: {movieFilename}')
imgList=[]
for img_file in fnmatch.filter(os.listdir(img_dir),'*.png'): 
    imgList.append((os.path.join(img_dir, img_file)))
imgList.sort()

imagesArray = []
for imageFile in imgList:
    # print(f'adding {imageFile} to imagesArray.')
    imagesArray.append(imageio.imread(imageFile))
imageio.mimsave(movieFilename, imagesArray, duration=0.1)
print ('<----- Done. ----->\n')

Creating gif movie: Z:\LWI_StageIV\!Accumulated\test-accum.gif
<----- Done. ----->

